# Final project: 
# Step 6: Generating tour and extras

In [12]:
#importamos librerias
import pandas as pd
import folium

### 1. Map with the recommended cities 

In [224]:
#ejemplo de lista de recomendacion
'''rec = ['Bangkok, Thailand',
 'Valencia, Spain',
 'Boulder, CO, United States',
 'Budapest, Hungary',
 'Geneva, Switzerland',
 'Cannes, France',
 'Amsterdam, Netherlands',
 'Vienna, Austria',
 'London, United Kingdom',
 'Beijing, China']
'''
#rec = ['Los Angeles, CA, United States', 'Hong Kong, Hong Kong', 'Seoul, South Korea', 'Singapore, Singapore', 'London, United Kingdom', 'Panama City, Panama', 'Guangzhou, China', 'San Miguel de Allende, Mexico', 'Cape Town, South Africa', 'Ho Chi Minh City, Vietnam']
rec = ['Queretaro, Mexico', 'Guadalajara, Mexico', 'Bangkok, Thailand', 'Hanoi, Vietnam', 'Mexico City, Mexico', 'Puerto Vallarta, Mexico', 'Chiang Mai, Thailand', 'Chiang Rai, Thailand']

In [225]:
#cargamos la tabla con todas las ciudades
cities = pd.read_csv('../data/nomadlist_cities.csv')

In [226]:
#función para recortar imagenes para mapa de folium
def crop_image(url,name):
    import requests
    from io import BytesIO
    import numpy as np
    from PIL import Image, ImageDraw

    response = requests.get(url)
    im = Image.open(BytesIO(response.content))
    width, height = im.size   # Get dimensions

    if height<width:
        new_s = height
    else:
        new_s = width

    left = (width - new_s)/2
    top = (height - new_s)/2
    right = (width + new_s)/2
    bottom = (height + new_s)/2

    # Crop the center of the image
    img = im.crop((left, top, right, bottom))
    
    # Open the input image as numpy array, convert to RGB
    npImage=np.array(img)
    h,w=img.size

    # Create same size alpha layer with circle
    alpha = Image.new('L', img.size,0)
    draw = ImageDraw.Draw(alpha)
    draw.pieslice([0,0,h,w],0,360,fill=255)

    # Convert alpha Image to numpy array
    npAlpha=np.array(alpha)

    # Add alpha layer to RGB
    npImage=np.dstack((npImage,npAlpha))

    # Save with alpha
    path = f'../Images/Cities/{name}.png'
    image_crop = Image.fromarray(npImage).save(path)
    
    return path

In [229]:
#función para generar mapa con las ciudades recomendadas
def mapa_reco(lst):
    #cargamos la tabla con todas las ciudades
    cities = pd.read_csv('../data/nomadlist_cities.csv')

    #generamos data frame con las ciudades recomendadas
    reco_cities = cities[cities['City-Country'].isin(lst)].reset_index(drop=True)

    #generamos mapa
    mapa=folium.Map([31.653830718,9.17416597],zoom_start=2)

    #pintamos las imagenes en los marcadores de cada ciudad
    for i in range(len(reco_cities)):
        try:
            icon = folium.features.CustomIcon(icon_image=f"../Images/Cities/{reco_cities.loc[i,'City-Country']}.png" ,icon_size=(80,80))
        except:
            icon_path = crop_image(reco_cities.loc[i,'Photo'],reco_cities.loc[i,'City-Country'])
            icon = folium.features.CustomIcon(icon_image=icon_path ,icon_size=(80,80))

        folium.Marker([reco_cities.loc[i,"lat"],reco_cities.loc[i,"lng"]],
                      icon=icon,
                      tooltip = reco_cities.loc[i,'City-Country'],
                      popup = f"Travelers City Score: {reco_cities.loc[i,'Overall Score']}/5").add_to(mapa)

    return mapa

In [230]:
#prueba de función
mapa_reco(rec)

### 2. Recommended route

In [231]:
start = rec[0]

In [232]:
ruta = cities[cities['City-Country']==start].reset_index(drop=True)

In [233]:
import ast 

ruta = cities[cities['City-Country']==start].reset_index(drop=True)

indice = 0
while len(ruta)<10:
    print(indice)
    print(ruta.loc[indice,'City-Country'])
    cerca = list(ast.literal_eval(ruta.loc[indice,'Closest Cities (km)']).keys())
    print(cerca)
    if len(ruta)<2:
        for i in cerca:
            if i not in list(ruta['City-Country']):
                print('se selecciona:',i)
                print('-----')
                ruta = ruta.append(cities[cities['City-Country']==i]).reset_index(drop=True)
                break
    else:
        for i in cerca:
            if i not in list(ruta['City-Country']):
                sentido = ruta.loc[indice-1,'lng']<ruta.loc[indice,'lng'] #si es true, va a la derecha
                lng_i = list(cities[cities['City-Country']==i]['lng'].values)[0]
                if sentido:
                    if ruta.loc[indice,'lng']<lng_i:
                        print('se selecciona:',i)
                        print('-----')
                        ruta = ruta.append(cities[cities['City-Country']==i]).reset_index(drop=True)
                        break
                else:
                    if ruta.loc[indice,'lng']>lng_i:
                        print('se selecciona:',i)
                        print('-----')
                        ruta = ruta.append(cities[cities['City-Country']==i]).reset_index(drop=True)
                        break
    indice+=1

0
Queretaro, Mexico
['San Miguel de Allende, Mexico', 'Guanajuato, Mexico', 'Toluca, Mexico', 'San Luis Potosi, Mexico', 'Mexico City, Mexico']
se selecciona: San Miguel de Allende, Mexico
-----
1
San Miguel de Allende, Mexico
['Queretaro, Mexico', 'Guanajuato, Mexico', 'San Luis Potosi, Mexico', 'Aguascalientes, Mexico', 'Toluca, Mexico']
se selecciona: Guanajuato, Mexico
-----
2
Guanajuato, Mexico
['San Miguel de Allende, Mexico', 'Queretaro, Mexico', 'San Luis Potosi, Mexico', 'Aguascalientes, Mexico', 'Guadalajara, Mexico']
se selecciona: Aguascalientes, Mexico
-----
3
Aguascalientes, Mexico
['San Luis Potosi, Mexico', 'Guanajuato, Mexico', 'Guadalajara, Mexico', 'San Miguel de Allende, Mexico', 'Queretaro, Mexico']
se selecciona: Guadalajara, Mexico
-----
4
Guadalajara, Mexico
['Aguascalientes, Mexico', 'Puerto Vallarta, Mexico', 'Sayulita, Mexico', 'Guanajuato, Mexico', 'San Miguel de Allende, Mexico']
se selecciona: Puerto Vallarta, Mexico
-----
5
Puerto Vallarta, Mexico
['Sayul

KeyError: 7

In [234]:
ruta

,City,Country,Photo,Overall Score,Quality of life score,Family score,Cost/month (USD),Internet Mbps,Fun,Safety,...,Temperature (°C) Aug,Temperature (°C) Sep,Temperature (°C) Oct,Temperature (°C) Nov,Temperature (°C) Dec,lat,lng,Closest Cities (km),Continente,City-Country
0,Queretaro,Mexico,https://nomadlist.com/assets/img/places/queret...,3.62,3,3,989.0,10.0,4.0,Good,...,24,24,23,23,21,20.595471,-100.397059,"{'San Miguel de Allende, Mexico': 50, 'Guanaju...",North America,"Queretaro, Mexico"
1,San Miguel de Allende,Mexico,https://nomadlist.com/assets/img/places/san-mi...,3.38,3,3,1211.0,11.0,3.0,Good,...,24,23,22,22,20,20.913902,-100.744012,"{'Queretaro, Mexico': 50, 'Guanajuato, Mexico'...",North America,"San Miguel de Allende, Mexico"
2,Guanajuato,Mexico,https://nomadlist.com/assets/img/places/guanaj...,3.43,3,3,1224.0,14.0,3.0,Okay,...,23,23,22,21,20,21.015885,-101.252841,"{'San Miguel de Allende, Mexico': 54, 'Quereta...",North America,"Guanajuato, Mexico"
3,Aguascalientes,Mexico,https://nomadlist.com/assets/img/places/aguasc...,3.12,3,3,2680.0,10.0,2.0,Okay,...,25,24,23,22,20,21.880655,-102.296132,"{'San Luis Potosi, Mexico': 139, 'Guanajuato, ...",North America,"Aguascalientes, Mexico"
4,Guadalajara,Mexico,https://nomadlist.com/assets/img/places/guadal...,3.60,3,3,1011.0,12.0,3.0,Okay,...,26,25,25,24,23,20.672037,-103.338396,"{'Aguascalientes, Mexico': 172, 'Puerto Vallar...",North America,"Guadalajara, Mexico"
5,Puerto Vallarta,Mexico,https://nomadlist.com/assets/img/places/puerto...,4.06,4,3,1352.0,22.0,3.0,Good,...,31,30,30,30,28,20.640718,-105.220307,"{'Sayulita, Mexico': 34, 'Guadalajara, Mexico'...",North America,"Puerto Vallarta, Mexico"
6,Sayulita,Mexico,https://nomadlist.com/assets/img/places/sayuli...,3.28,3,3,2096.0,15.0,3.0,Okay,...,31,31,30,29,27,20.868926,-105.440702,"{'Puerto Vallarta, Mexico': 34, 'Guadalajara, ...",NaN,"Sayulita, Mexico"


In [235]:
url = 'https://www.google.com.mx/maps/dir'
for i in ruta['City-Country']:
    url = f'{url}/{i}'.replace(' ','+')
url    

'https://www.google.com.mx/maps/dir/Queretaro,+Mexico/San+Miguel+de+Allende,+Mexico/Guanajuato,+Mexico/Aguascalientes,+Mexico/Guadalajara,+Mexico/Puerto+Vallarta,+Mexico/Sayulita,+Mexico'